# [Day2-2] 대용량 처리: 이터레이터와 제너레이터

`학습 목표`

1. `이터레이터 이해`: `iter()`와 `next()`의 동작 원리를 이해하고 활용합니다.

2. `제너레이터 활용`: `yield`를 사용한 지연 평가(Lazy Evaluation)로 메모리를 절약합니다.
3. `제너레이터 표현식`: 한 줄로 메모리 효율적인 데이터 처리 파이프라인을 구축합니다.
4. `실무 응용`: 대용량 로그 파일을 메모리에 모두 올리지 않고 스트리밍 방식으로 처리합니다.

---

---

## Part 1: 이터레이터와 제너레이터 기초

이터레이터의 동작 원리와 제너레이터의 기본 개념을 학습합니다.

---

## 1. [Iterators] 이터레이터: 순회 가능한 객체의 비밀

### 1.1 이터레이터란?

`이터레이터(Iterator)`는 데이터를 순차적으로 꺼낼 수 있는 객체입니다. 

* `for` 루프가 작동하는 원리가 바로 이터레이터입니다.

* 모든 시퀀스(리스트, 튜플, 문자열)는 `iter()` 함수로 이터레이터로 변환할 수 있습니다.
* `next()` 함수로 다음 요소를 하나씩 꺼냅니다.
* 더 이상 꺼낼 요소가 없으면 `StopIteration` 예외가 발생합니다.

In [ ]:
# [기본] 리스트를 이터레이터로 변환
numbers = [1, 2, 3]
iterator = iter(numbers)

print(type(numbers))

<class 'list'>


In [2]:
print(type(iterator))

<class 'list_iterator'>


In [3]:
# next()로 하나씩 꺼내기
print(next(iterator))  # 1

1


In [4]:
print(next(iterator))  # 2

2


In [5]:
print(next(iterator))  # 3

3


In [ ]:
# 더 이상 꺼낼 요소가 없으면 StopIteration 예외 발생!
print(next(iterator))  

StopIteration: 

### 1.2 for 루프와 이터레이터

`for` 루프는 내부적으로 `iter()`와 `next()`를 사용합니다.

In [7]:
# 이 for 루프는...
products = ["Mouse", "Keyboard", "Monitor"]
for product in products:
    print(product)

Mouse
Keyboard
Monitor


In [8]:
# 내부적으로 이렇게 동작합니다
products = ["Mouse", "Keyboard", "Monitor"]
it = iter(products)

while True:
    try:
        product = next(it)
        print(product)
    except StopIteration:
        break  # 더 이상 요소가 없으면 종료

Mouse
Keyboard
Monitor


---

## 2. [Generators] 제너레이터: 메모리를 아끼는 마법

### 2.1 제너레이터란?

`제너레이터(Generator)`는 `yield` 키워드를 사용하는 특별한 함수입니다.

* 일반 함수는 `return`으로 값을 반환하고 종료됩니다.

* 제너레이터는 `yield`로 값을 하나씩 "던져주고" 일시 중지됩니다.
* 다음 `next()` 호출 시 중단된 지점부터 다시 실행됩니다.
* **메모리에 모든 데이터를 올리지 않고** 필요할 때만 하나씩 생성합니다.

In [9]:
# [실습 1] 간단한 제너레이터 함수
def count_up_to(n):
    """1부터 n까지 하나씩 생성하는 제너레이터"""
    count = 1
    while count <= n:
        yield count  # 값을 반환하고 일시 중지
        count += 1


In [ ]:
# 제너레이터 객체 생성
counter = count_up_to(3)
print(type(counter)) 

<class 'generator'>


In [11]:
# next()로 하나씩 꺼내기
print(next(counter))  # 1

1


In [12]:
print(next(counter))  # 2

2


In [ ]:
print(next(counter))  # 3

3


In [ ]:
print(next(counter))  # StopIteration 발생

StopIteration: 

In [15]:
# for 루프에서 자동으로 처리
for num in count_up_to(5):
    print(f"숫자: {num}")

숫자: 1
숫자: 2
숫자: 3
숫자: 4
숫자: 5


### 2.2 제너레이터 vs 리스트: 메모리 효율성

**중요:** 제너레이터는 값을 미리 생성하지 않습니다!

In [16]:
# [비교] 리스트 방식 (메모리에 모두 저장)
def get_squares_list(n):
    """1부터 n까지의 제곱수 리스트 반환"""
    result = []
    for i in range(1, n + 1):
        result.append(i ** 2)
    return result

squares_list = get_squares_list(1000000)
# 100만 개의 숫자를 메모리에 모두 저장!

In [28]:
import sys
# 리스트는 메모리에 연속적으로 저장되어 있어 메모리 사용량이 많음
print(f"{int(sys.getsizeof(squares_list) / 1024)} KB")

8250 KB


In [23]:
# [비교] 제너레이터 방식 (필요할 때만 생성)
def get_squares_gen(n):
    """1부터 n까지의 제곱수 제너레이터"""
    for i in range(1, n + 1):
        yield i ** 2

squares_gen = get_squares_gen(1000000)
# 제너레이터 객체만 생성! 실제 값은 아직 만들어지지 않음

In [24]:
# 제너레이터는 필요한 것만 꺼내 씁니다
for square in get_squares_gen(5):
    print(square)  # 1, 4, 9, 16, 25 - 하나씩 계산되어 출력됨

1
4
9
16
25


In [27]:
# Generator는 메모리 사용량이 적음
print(f"{int(sys.getsizeof(squares_gen))} Byte")

216 Byte


### 2.3 실무 예제: 대용량 로그 파일 읽기

수 기가바이트의 로그 파일을 메모리에 모두 올리면 시스템이 멈춥니다. 제너레이터로 한 줄씩 읽어 처리합니다.

In [34]:
!python log_generator.py

[server.log] 생성 시작... (총 25,000,000줄, 예상 크기: 1.05 GB)
진행률: 10% 완료 (2,500,000줄, 소요 시간: 6초)
진행률: 20% 완료 (5,000,000줄, 소요 시간: 13초)
진행률: 30% 완료 (7,500,000줄, 소요 시간: 20초)
진행률: 40% 완료 (10,000,000줄, 소요 시간: 27초)
진행률: 50% 완료 (12,500,000줄, 소요 시간: 34초)
진행률: 60% 완료 (15,000,000줄, 소요 시간: 41초)
진행률: 70% 완료 (17,500,000줄, 소요 시간: 48초)
진행률: 80% 완료 (20,000,000줄, 소요 시간: 55초)
진행률: 90% 완료 (22,500,000줄, 소요 시간: 62초)
진행률: 100% 완료 (25,000,000줄, 소요 시간: 69초)
생성 완료: /Users/dante/workspace/dante-code/class/statrack2_python/week1/server.log


In [35]:
!du -h server.log

1.3G	server.log


In [36]:
# [실무 예제] 로그 파일 스트리밍 처리
def read_large_file(file_path):
    """
    대용량 파일을 한 줄씩 읽어 반환하는 제너레이터
    
    메모리에 전체 파일을 올리지 않고,
    필요할 때마다 한 줄씩 읽어 처리합니다.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:  # 파일은 이미 이터레이터입니다!
            yield line.strip()

In [ ]:
for log_line in read_large_file('server.log'):
    if 'ERROR' in log_line:
        print(f"에러 발견: {log_line}")

---

## Part 2: 제너레이터 고급 활용

제너레이터 표현식을 활용한 메모리 효율적인 데이터 처리 파이프라인을 구축합니다.

---

---

## 3. [Generator Expressions] 제너레이터 표현식

### 3.1 리스트 컴프리헨션 vs 제너레이터 표현식

* **리스트 컴프리헨션**: `[ ]` 사용 → 리스트 생성
* **제너레이터 표현식**: `( )` 사용 → 제너레이터 생성

문법은 거의 동일하지만, 메모리 사용량이 완전히 다릅니다!

In [ ]:
# [비교] 리스트 컴프리헨션 - 즉시 실행
squares_list = [x ** 2 for x in range(10)]
print(type(squares_list))
print(squares_list)

<class 'list'>
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [39]:
# [비교] 제너레이터 표현식 - 지연 평가
squares_gen = (x ** 2 for x in range(10))
print(type(squares_gen))
print(squares_gen)         # <generator object ...> - 아직 계산되지 않음!

<class 'generator'>
<generator object <genexpr> at 0x13a83cba0>


In [40]:
# 제너레이터 표현식은 필요할 때 꺼내 씁니다
for square in squares_gen:
    print(square)

0
1
4
9
16
25
36
49
64
81


### 3.2 실무 활용: 데이터 필터링 파이프라인

In [41]:
# [실무 예제] 매출 데이터 필터링
sales_data = [
    {"product": "Mouse", "price": 15000, "quantity": 100},
    {"product": "Keyboard", "price": 45000, "quantity": 50},
    {"product": "Monitor", "price": 250000, "quantity": 20},
    {"product": "Cable", "price": 5000, "quantity": 200}
]
sales_data

[{'product': 'Mouse', 'price': 15000, 'quantity': 100},
 {'product': 'Keyboard', 'price': 45000, 'quantity': 50},
 {'product': 'Monitor', 'price': 250000, 'quantity': 20},
 {'product': 'Cable', 'price': 5000, 'quantity': 200}]

In [42]:
# 총 매출 100만원 이상인 제품만 필터링 (제너레이터 표현식)
high_revenue_products = (
    item for item in sales_data 
    if item["price"] * item["quantity"] >= 1000000
)
high_revenue_products

<generator object <genexpr> at 0x13a83c6c0>

In [43]:
for product in high_revenue_products:
    revenue = product["price"] * product["quantity"]
    print(f"{product['product']}: {revenue:,}원")

Mouse: 1,500,000원
Keyboard: 2,250,000원
Monitor: 5,000,000원
Cable: 1,000,000원


### 3.3 함수와 함께 사용하기

`sum()`, `max()`, `min()` 등의 함수는 제너레이터를 직접 받을 수 있습니다.

In [44]:
# [실습] 1부터 100까지의 합 (제너레이터 표현식)
total = sum(x for x in range(1, 101))
print(f"1부터 100까지의 합: {total}")

1부터 100까지의 합: 5050


In [45]:
# [실습] 짝수만 추출하여 최댓값 구하기
numbers = [15, 22, 8, 34, 7, 19, 40]
max_even = max(x for x in numbers if x % 2 == 0)
print(f"가장 큰 짝수: {max_even}")

가장 큰 짝수: 40


---

## 4. 2교시 실전 퀴즈 (Quizzes)

지금까지 배운 이터레이터, 제너레이터, 제너레이터 표현식을 모두 활용해 보세요.

### Q1. [이터레이터] 리스트 수동 순회 (난이도: ⭐⭐)

다음 데이터가 주어졌습니다.
`items = ["Apple", "Banana", "Cherry"]`

1. `iter()` 함수로 `items`를 이터레이터로 변환하세요.

2. `next()` 함수를 **3번** 호출하여 각 과일 이름을 출력하세요.
3. 한 번 더 `next()`를 호출하면 어떤 예외가 발생하는지 주석으로 작성하세요.

In [ ]:
items = ["Apple", "Banana", "Cherry"]

# [Q1 풀이 작성]
# 1. 이터레이터 생성

# 2. next() 3번 호출하여 출력

# 3. 한 번 더 호출 시 발생하는 예외: 

### Q2. [이터레이터] 수동 순회와 합계 계산 (난이도: ⭐⭐)

숫자 리스트가 주어졌습니다.
`numbers = [10, 20, 30, 40, 50]`

`함수 작성:`
* `sum_with_iterator(numbers)` 함수를 작성하세요.

* `iter()`와 `next()`를 사용하여 리스트의 모든 요소를 더한 합계를 반환합니다.
* `for` 루프를 사용하지 말고, `while` 루프와 `try-except`로 `StopIteration`을 처리하세요.

#### 예시
```python
result = sum_with_iterator([10, 20, 30])
print(result)  # Output: 60
```

In [ ]:
# [Q2 풀이 작성]
def sum_with_iterator(numbers):
    """
    이터레이터를 사용하여 리스트의 합계를 계산합니다.
    
    Args:
        numbers (list): 숫자 리스트
    
    Returns:
        int/float: 모든 요소의 합
    """
    # Your code here
    pass

# 테스트 코드
# result = sum_with_iterator([10, 20, 30, 40, 50])
# print(result)  # 예상 결과: 150

### Q3. [제너레이터 표현식] 짝수만 추출 (난이도: ⭐⭐⭐)

1부터 20까지의 숫자 중 **짝수만** 추출하는 제너레이터 표현식을 작성하세요.

1. 제너레이터 표현식을 사용하여 `even_gen`을 생성하세요.

2. `list()` 함수로 제너레이터를 리스트로 변환하여 출력하세요.
3. 같은 작업을 리스트 컴프리헨션으로도 작성하고, 차이점을 주석으로 설명하세요.

In [ ]:
# [Q3 풀이 작성]
# 1. 제너레이터 표현식으로 짝수 생성

# 2. 리스트로 변환하여 출력

# 3. 리스트 컴프리헨션으로도 작성

# 차이점: 

### Q4. [제너레이터] 피보나치 수열 생성기 (난이도: ⭐⭐⭐)

`함수 작성:`
* `fibonacci_gen(n)` 제너레이터 함수를 작성하세요.

* 피보나치 수열의 처음 `n`개를 `yield`로 반환합니다.
* 피보나치 수열: 0, 1, 1, 2, 3, 5, 8, 13, 21, ...
  (각 숫자는 앞 두 숫자의 합입니다)

#### 예시
```python
for num in fibonacci_gen(7):
    print(num, end=" ")  # Output: 0 1 1 2 3 5 8
```

In [ ]:
# [Q4 풀이 작성]
def fibonacci_gen(n):
    """
    피보나치 수열의 처음 n개를 생성하는 제너레이터
    
    Args:
        n (int): 생성할 피보나치 수의 개수
    
    Yields:
        int: 피보나치 수
    """
    # Your code here
    pass

# 테스트 코드
# for num in fibonacci_gen(10):
#     print(num, end=" ")  # 예상 결과: 0 1 1 2 3 5 8 13 21 34

### Q5. [제너레이터] 범위 내 데이터 스트리밍 (난이도: ⭐⭐⭐⭐)

매출 데이터를 처리하는 제너레이터를 작성합니다.

`함수 작성:`
* `filter_sales(sales_data, min_amount)` 제너레이터 함수를 작성하세요.

* `sales_data`는 딕셔너리 리스트입니다: `[{"product": "A", "amount": 100}, ...]`
* `amount`가 `min_amount` 이상인 데이터만 `yield`로 반환합니다.
* 반환 시 `amount`를 1,000원 단위로 반올림하여 반환합니다. (힌트: `round(amount, -3)`)

#### 예시
```python
data = [
    {"product": "A", "amount": 15500},
    {"product": "B", "amount": 8300},
    {"product": "C", "amount": 25700}
]
for item in filter_sales(data, 10000):
    print(item)  
# Output: 
# {'product': 'A', 'amount': 16000}
# {'product': 'C', 'amount': 26000}
```

In [ ]:
# [Q5 풀이 작성]
def filter_sales(sales_data, min_amount):
    """
    최소 금액 이상의 매출 데이터를 필터링하고 반올림하여 반환
    
    Args:
        sales_data (list): 매출 데이터 딕셔너리 리스트
        min_amount (int): 최소 금액
    
    Yields:
        dict: 필터링되고 반올림된 매출 데이터
    """
    # Your code here
    pass

# 테스트 코드
# test_data = [
#     {"product": "A", "amount": 15500},
#     {"product": "B", "amount": 8300},
#     {"product": "C", "amount": 25700}
# ]
# for item in filter_sales(test_data, 10000):
#     print(item)

### Q6. [제너레이터] 파일 읽기 시뮬레이션 (난이도: ⭐⭐⭐⭐)

실제 파일 대신 문자열 리스트로 로그 데이터를 시뮬레이션합니다.

`함수 작성:`
* `parse_log_lines(log_lines)` 제너레이터 함수를 작성하세요.

* `log_lines`는 문자열 리스트입니다: `["INFO:Started", "ERROR:Failed", ...]`
* 각 로그 라인을 `:`로 분리하여 `(레벨, 메시지)` 튜플로 `yield`합니다.
* 빈 줄은 건너뜁니다.

#### 예시
```python
logs = [
    "INFO:Application started",
    "ERROR:Connection failed",
    "",
    "WARN:Low disk space"
]
for level, message in parse_log_lines(logs):
    print(f"[{level}] {message}")
# Output:
# [INFO] Application started
# [ERROR] Connection failed
# [WARN] Low disk space
```

In [ ]:
# [Q6 풀이 작성]
def parse_log_lines(log_lines):
    """
    로그 라인을 파싱하여 (레벨, 메시지) 튜플로 반환
    
    Args:
        log_lines (list): 로그 문자열 리스트
    
    Yields:
        tuple: (레벨, 메시지)
    """
    # Your code here
    pass

# 테스트 코드
# test_logs = [
#     "INFO:Application started",
#     "ERROR:Connection failed",
#     "",
#     "WARN:Low disk space",
#     "INFO:Process completed"
# ]
# for level, message in parse_log_lines(test_logs):
#     print(f"[{level}] {message}")

### Q7. [종합/심화] 대용량 로그 분석 시스템 (난이도: ⭐⭐⭐⭐⭐)

여러 제너레이터를 조합하여 대용량 로그 분석 시스템을 구축합니다.

다음 로그 데이터가 주어졌습니다:
```python
server_logs = [
    "2025-01-01 10:00:00 INFO User logged in",
    "2025-01-01 10:05:23 ERROR Database connection failed",
    "2025-01-01 10:10:45 WARN High memory usage: 85%",
    "2025-01-01 10:15:12 ERROR Payment processing failed",
    "2025-01-01 10:20:33 INFO User logged out",
    "2025-01-01 10:25:55 ERROR API timeout"
]
```

`작업:`
1. `extract_errors(log_lines)` 제너레이터: "ERROR"가 포함된 로그만 `yield`

2. `extract_timestamp(error_lines)` 제너레이터: 로그에서 시간 부분(앞 19글자)만 추출하여 `yield`
3. 두 제너레이터를 **체이닝**하여 에러 발생 시간만 추출하고 출력하세요.
4. 총 에러 개수를 세어 출력하세요. (힌트: `sum(1 for _ in ...)`)

In [ ]:
server_logs = [
    "2025-01-01 10:00:00 INFO User logged in",
    "2025-01-01 10:05:23 ERROR Database connection failed",
    "2025-01-01 10:10:45 WARN High memory usage: 85%",
    "2025-01-01 10:15:12 ERROR Payment processing failed",
    "2025-01-01 10:20:33 INFO User logged out",
    "2025-01-01 10:25:55 ERROR API timeout"
]

# [Q7 풀이 작성]
# 1. 에러 로그만 추출하는 제너레이터
def extract_errors(log_lines):
    """ERROR가 포함된 로그만 반환"""
    # Your code here
    pass

# 2. 타임스탬프만 추출하는 제너레이터
def extract_timestamp(error_lines):
    """로그에서 시간 부분(앞 19글자)만 추출"""
    # Your code here
    pass

# 3. 제너레이터 체이닝하여 에러 발생 시간 출력

# 4. 총 에러 개수 세기

---

## 🎯 학습 정리

### 핵심 개념 복습

| 개념 | 문법/함수 | 사용 예시 |
|------|----------|----------|
| 이터레이터 생성 | `iter(sequence)` | `it = iter([1,2,3])` |
| 다음 요소 꺼내기 | `next(iterator)` | `next(it)` → `1` |
| 제너레이터 함수 | `yield` | `yield x` - 값 반환 후 일시 중지 |
| 제너레이터 표현식 | `(expr for ...)` | `(x*2 for x in range(10))` |
| 리스트 컴프리헨션 | `[expr for ...]` | `[x*2 for x in range(10)]` |
| 메모리 효율 | 제너레이터 사용 | 큰 데이터는 제너레이터로! |

### Why 제너레이터?

✅ **메모리 효율**: 100만 개 데이터를 한 번에 메모리에 올리지 않음

✅ **지연 평가**: 필요할 때만 계산 (Lazy Evaluation)

✅ **대용량 처리**: 수 GB 로그 파일도 스트리밍 방식으로 처리

✅ **파이프라인**: 여러 제너레이터를 체이닝하여 데이터 처리 흐름 구축